# House Prices - Advanced Regression Techniques (Detailed Project Documentation)
---


1. [Introduction and Objectives](#introduction)
2. [Data Loading and Initial Exploration](#data-loading)
3. [Exploratory Data Analysis](#eda)
4. [Feature Engineering and Custom Transformers](#feature-engineering)
5. [Statistical Outlier Detection and Z-Score Analysis](#outlier-detection)
6. [Data Preprocessing Pipeline](#preprocessing)
7. [Model Implementation and Baseline](#modeling)
8. [Automated Hyperparameter Optimization with Optuna](#optimization)
9. [Model Evaluation with Cross-Validation and Ensemble Analysis](#evaluation)
10. [Predictions and Kaggle Submission](#submission)


## Introduction

This notebook implements a production-ready machine learning pipeline for predicting house prices using the Kaggle House Prices dataset. The approach emphasizes automated optimization, ensemble learning, and robust evaluation practices suitable for real-world deployment. Involves statistical analysis, data preprocessing, feature engineering, costumized transformers/pipelines and stacking regression. Finally: model evaluation and selection via Optuna Hyperparameter Tuning


### Technical Implementation Strategy

The solution employs automated ensemble discovery where Optuna simultaneously optimizes model selection and hyperparameters across 9 different algorithm types. This eliminates manual model tuning while ensuring robust cross-validation evaluation. The final stacking architecture combines diverse base learners (linear, tree-based, kernel, instance-based) using a meta-learner that optimally weights predictions.

---

### Methodology
This project tackles house price prediction through ensemble learning pipeline that automates the entire model selection and optimization process.

**Custom Feature Engineering Pipeline**
- Built 8 custom sklearn transformers handling log transformations, age calculations, outlier removal, categorical encoding, and correlation filtering
- Created binary indicators for sparse features (like ScreenPorch, MasVnrArea) to capture zero/non-zero patterns
- Combined basement and ground floor areas into unified TotalSF feature
- Weighted bathroom counting (full baths = 1.0, half baths = 0.5) for better representation
- One-hot encoded categorical features with high cardinality (inspected manually) and non-ordinal. 
- Ordinal encoded categorical features with low cardinality (inspected manually).

**Outlier Detection System**
- Implemented z-score statistical analysis across all numerical features
- Generated comprehensive outlier tables ranking extreme values by magnitude
- Applied systematic outlier removal (|z| > 3.0) only on training data to prevent data leakage
- Preserved outliers in validation/test sets for realistic performance evaluation

**Automated Model Discovery with Optuna**
- 35/50 trial optimization discovering optimal ensemble compositions automatically
- Simultaneous hyperparameter tuning for 9 different algorithm types (Linear, Ridge, Lasso, ElasticNet, CatBoost, SVR, KNN, RandomForest, LightGBM, XGBoost)
- Dynamic model selection flags allowing Optuna to choose which algorithms to include
- Cross-validation scoring ensuring robust model evaluation

**Multi-Algorithm Stacking Architecture**
- Final ensemble combines 5 diverse base learners: Lasso, CatBoost, SVR, RandomForest, XGBoost
- Ridge regression meta-learner optimally weights base model predictions
- 5-fold cross-validation prevents overfitting during stacking
- Leverages different algorithm strengths: linear relationships (Lasso), non-linear patterns (trees), local patterns (SVR)

**Data Pipeline**
- Separate pipelines for train (with outlier removal) vs validation/test (without outlier removal)
- Consistent feature engineering across all datasets using fitted transformers
- DataFrame alignment
- Maintained pipeline order preventing any form of data leakage

**Model Evaluation**
- RMSE, R², and MAE metrics for complete performance assessment
- Residual analysis 
- Features importance analysis using SHAP values
- Optuna visualization tools showing optimization history and parameter importance
---

In [ ]:
# Core libraries and type hints for enhanced code clarity
from typing import List, Optional, Union, Dict, Any

In [ ]:
# Essential libraries for data manipulation, visualization, and machine learning
import pandas as pd
import torch 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading training and test datasets from Kaggle competition
test_data = pd.read_csv('data/test.csv')
train_data = pd.read_csv('data/train.csv')

In [ ]:
# Exploring dataset structure by examining all available features
sorted_columns = sorted(train_data.columns)
for column in sorted_columns:
    print(column)

In [ ]:
train_data.info()

In [ ]:
# Creating train-validation split for model evaluation
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['SalePrice'], axis=1).copy()
y = train_data['SalePrice'].copy()

# Splitting data with 80-20 ratio, maintaining reproducibility
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reconstructing training data for pipeline processing
train_data = X_train.join(y_train)

In [ ]:
X_val.info()

In [ ]:
# Analyzing target variable distribution to understand data characteristics
sns.histplot(train_data['SalePrice'], bins=100, kde = True, linewidth=0)
plt.xlabel('Sale Price')

In [ ]:
# Extracting numerical features for comprehensive analysis
numerical_values = train_data.select_dtypes(include=[np.number])

In [ ]:
# Creating comprehensive histogram visualization for all numerical features
numerical_values.hist(figsize=(15, 10), bins=50, xlabelsize=8, ylabelsize=8)
for ax in plt.gcf().axes:
    ax.set_title(ax.get_title(), fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
numerical_values.info() 

## Feature Engineering and Custom Transformers 

### Log Transformation Strategy

Many numerical features exhibit significant right-skewness, particularly area-based measurements like LotArea, GrLivArea, and TotalBsmtSF. Applying log1p transformation reduces skewness and creates more Gaussian-like distributions, which enhances model performance for algorithms that assume normality.

The transformation also helps stabilize variance across different scales, making the data more suitable for linear models and improving convergence in gradient-based optimization algorithms. 
We apply the log transformation only on those features that are appear from continous from the manual investigation. Those features were selected one by one. 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# Custom transformers implementing sklearn interface for systematic preprocessing

class LogTransformer(BaseEstimator, TransformerMixin):
    """Applies logarithmic transformation to reduce skewness in numerical features.
    Creates binary indicators for sparse features with high zero proportions."""
    
    def __init__(self, min_prop = 0.05, columns =["LotArea", "GrLivArea",
                                "TotalBsmtSF", "1stFlrSF", "LotFrontage",
                                "GarageArea", "ScreenPorch",
                                "BsmtUnfSF", "MasVnrArea", "BsmtFinSF1",
                                "BsmtFinSF2", "OpenPorchSF"]):
        self.min_prop = min_prop
        self.columns = columns

    def fit(self, X, y=None):
        # Identifying sparse columns for binary indicator creation
        self.sparse_columns = []
        for col in self.columns:
            value_counts = X[col].value_counts(normalize=True)
            zero_prop = value_counts.get(0, 0.0)
            if zero_prop >= self.min_prop: 
                self.sparse_columns.append(col)
        return self

    def transform(self, X):
        X = X.copy()
        if self.columns is not None:
            for col in self.columns:
                X[col] = np.log1p(X[col])
                if col in self.sparse_columns:
                    X[f"has_{col}"] = (X[col] > 0).astype(int)
        return X

class dropColumns(BaseEstimator, TransformerMixin):
    """Removes specified columns that provide minimal predictive value or cause issues."""
    
    def __init__(self, columns=[]):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            if col in X.columns:
                X.drop(columns=col, inplace=True)
        return X

class AgeCalculator(BaseEstimator, TransformerMixin):
    """Computes house age from year sold and year built, providing temporal context."""
    
    def __init__(self, columns=["YrSold", "YearBuilt"]):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["HouseAge"] = (X[self.columns[0]] - X[self.columns[1]]).astype(int)
        X["HouseAge"] = X["HouseAge"].clip(lower=0)
        return X

class FeatureImputer(BaseEstimator, TransformerMixin):
    """Handles missing values with domain-specific strategies for different feature types."""
    
    def __init__(self, columns_zeros=[], columns_none = []):
        self.columns_zeros = columns_zeros
        self.columns_none = columns_none

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns_zeros:
            if col in X.columns:
                X[col] = X[col].fillna(0)
        for col in self.columns_none:
            if col in X.columns:
                X[col] = X[col].fillna("None")
        return X

class OutliersRemoval(BaseEstimator, TransformerMixin):
    """Removes predetermined outlier observations based on statistical analysis."""
    
    def __init__(self, rows_indexes =[1182, 1298, 1169, 224, 1190]):
        self.rows_indexes = rows_indexes 
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for index in self.rows_indexes:
            if index in X.index:
                X = X.drop(index=index, errors='ignore')
        return X.reset_index(drop=True)
    
class YearImputer(BaseEstimator, TransformerMixin):
    """Imputes missing garage year built values while creating missingness indicators."""
    
    def __init__(self, column="GarageYrBlt", fill_value= None):
        self.column = column
        self.fill_value = fill_value
       
    def fit(self, X, y=None):
        if self.fill_value is None:
            self.fill_value = X[self.column].median()
        return self

    def transform(self, X):
        X = X.copy()
        X["GarageYrBlt_missing"] = (X[self.column]).isna().astype(int)
        X[self.column] = X[self.column].fillna(self.fill_value)
        return X

class HighlyCorrelatedFeatures(BaseEstimator, TransformerMixin):
    """Identifies and removes highly correlated features to reduce multicollinearity."""
    
    def __init__(self, threshold=0.82):
        self.threshold = threshold
        self.columns_to_drop = []

    def fit(self, X, y=None):
        corr_matrix = X.corr().abs()
        mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        upper = corr_matrix.where(mask)
        self.to_drop_ = [col for col in upper.columns if any(upper[col] > self.threshold)]
        return self

    def transform(self, X):
        X = X.copy()
        return X.drop(columns=self.columns_to_drop, errors='ignore')

## Outlier Detection and Analysis

### Statistical outlier detection using z-score

Outliers can significantly impact model performance, particularly for linear algorithms. The z-score method provides a standardized approach to identify observations that deviate substantially from the dataset mean.

The z-score transformation standardizes values by measuring how many standard deviations an observation lies from the mean:

$$z_{i,j} = \frac{x_{i,j} - \mu_j}{\sigma_j}$$

Where:
- $\mu_j = \frac{1}{n}\sum_{i=1}^n x_{i,j}$ is the sample mean
- $\sigma_j = \sqrt{\frac{1}{n}\sum_{i=1}^n (x_{i,j} - \mu_j)^2}$ is the sample standard deviation

Observations with $|z_{i,j}| > 3$ are typically considered outliers and may warrant removal or special treatment. In this context, we apply z-score analysis to all numerical features, but we only remove outliers from the training set to prevent data leakage. The validation and test sets retain all observations, including outliers, to ensure realistic performance evaluation. Not every outlier needs to be removed, some of them could contain valuable information, especially in real estate where extreme values can indicate unique properties or market conditions. For this reason, we kept msot of the outliers and manually investigate which one to remove later on.



In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(numerical_values, title="House Prices Regression Data Profiling Report", explorative=True)
profile.to_file("house_prices_regression.html")

In the html file `house_prices_regression.html`, we can find the data profiling report which includes the Z-score for each value in the specified columns. The report also provides insights into the distribution of the data, missing values, and other statistical properties.



In [ ]:
from scipy.stats import zscore 

def get_outlier_table(numerical_values: pd.DataFrame, thresh: float = 3.0):
    """Creates a comprehensive outlier analysis table using Z-score methodology.
    Returns sorted table of extreme values for systematic outlier identification."""
    
    # Computing Z-scores for all numerical features
    zscores  = pd.DataFrame(zscore(numerical_values, nan_policy="omit"), 
                            columns=numerical_values.columns, 
                            index=numerical_values.index)

    # Reshaping to long format for analysis
    long = (zscores.stack().reset_index().rename(columns={
            "level_0": "row_index",
            "level_1": "feature",
            0: "z_score"}))

    # Filtering extreme values based on threshold
    outliers = long[long["z_score"].abs() > thresh].copy()
    
    # Sorting by magnitude for prioritized outlier review
    outliers = outliers.reindex(
        outliers["z_score"].abs().sort_values(ascending=False).index
    ).reset_index(drop=True)
    return outliers

# Generating outlier analysis for systematic review
outlier_table = get_outlier_table(numerical_values, thresh=3.0)

# Examining outliers in key features for decision-making
for feature in ["LotArea", "GrLivArea", "SalePrice",
                                "TotalBsmtSF", "1stFlrSF", "LotFrontage",
                                "GarageArea", "ScreenPorch",
                                "BsmtUnfSF", "MasVnrArea", "BsmtFinSF1",
                                "BsmtFinSF2", "OpenPorchSF"]:
    if feature in outlier_table["feature"].values: 
        print(outlier_table[outlier_table["feature"] == feature].head(10))

def get_outliers_list(outlier_table: pd.DataFrame, n: int = 10):
    """Extracts list of most extreme outlier indices for removal consideration."""
    return outlier_table.head(n)["row_index"].tolist()

In [ ]:
plt.scatter(x="SalePrice", y="BsmtFinSF1", data=train_data)

In [ ]:
train_data.query("SalePrice > 500000")
#1182, 1298, 1169

In [ ]:
plt.scatter(x="YrSold", y="SalePrice", data=train_data)

In [ ]:
pd.DataFrame(train_data.isnull().sum(), columns=['Null Count']).sort_values(by='Null Count', ascending=False).head(20)

### Categorical Feature Analysis

Examining categorical features to determine appropriate encoding strategies based on feature characteristics and relationships with the target variable.

In [ ]:
# We used this cell to manually inspect the unique values of different columns
train_data["GarageFinish"].unique()

In [ ]:
# We used this cell to manually fill some missing values in the dataset
train_data["BsmtFinType1"] = train_data["BsmtFinType1"].fillna("Unf")
train_data["BsmtFinType2"] = train_data["BsmtFinType2"].fillna("Unf")
train_data["Electrical"] = train_data["Electrical"].fillna("SBrkr")

X_val["BsmtFinType1"] = X_val["BsmtFinType1"].fillna("Unf")
X_val["BsmtFinType2"] = X_val["BsmtFinType2"].fillna("Unf")
X_val["Electrical"] = X_val["Electrical"].fillna("SBrkr")



In [ ]:
plt.scatter(x="MasVnrArea", y="LotFrontage", data=train_data)

In [ ]:
sns.catplot(x="GarageCond", y="SalePrice", data=train_data, kind="box")

In [ ]:
'''Inspecting categorical features and their unique values'''

# Get all categorical columns
categorical_columns = train_data.select_dtypes(include=['object']).columns

# Create a list to store dictionaries with column info
column_info = []

# Populate the list
for col in categorical_columns:
    unique_vals = train_data[col].unique()
    column_info.append({'Feature': col, 'Unique_Values': list(unique_vals)})

# Create DataFrame from the list
feature_values_df = pd.DataFrame(column_info)

# Display the resulting dataframe
# Set pandas display options to show all values without truncation
pd.set_option('display.max_colwidth', None)
feature_values_df.head(5)   

In [ ]:
# Categorizing features based on encoding requirements

# Nominal categorical variables requiring one-hot encoding
one_hot_features = ["SaleType", "MiscFeature", "PavedDrive", "GarageType", "Electrical", 
                    "CentralAir", "Heating", "Foundation", "MasVnrType", "RoofMatl", 
                    "Exterior2nd", "RoofStyle", "HouseStyle", "BldgType", "Exterior1st", 
                    "Condition2", "Condition1", "Neighborhood", "LotConfig", "LotShape", 
                    "Street", "MSZoning", "SaleCondition"]

# Ordinal categorical variables with inherent quality ordering
ordinal_features = ["GarageCond", "GarageQual", "GarageFinish", "FireplaceQu", 
                    "Functional", "KitchenQual", "HeatingQC", "BsmtFinType2", "BsmtFinType1", 
                    "BsmtExposure", "BsmtCond", "BsmtQual", "ExterCond", "ExterQual",
                    "LandSlope", "Utilities", "LandContour"]

In [ ]:
# Defining ordinal mappings with proper quality hierarchies from worst to best
ordinal_features_dict = { 
    "GarageCond":  ["Po", "Fa", "TA", "Gd", "Ex", "None"],
    "GarageQual":  ["Po", "Fa", "TA", "Gd", "Ex", "None"],
    "FireplaceQu": ["Po", "Fa", "TA", "Gd", "Ex", "None"],
    "KitchenQual": ["None","Po", "Fa", "TA", "Gd", "Ex"],
    "HeatingQC":   ["Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtCond":    ["Po", "Fa", "TA", "Gd", "Ex", "None"],
    "BsmtQual":    ["Po", "Fa", "TA", "Gd", "Ex", "None"],
    "ExterCond":   ["Po", "Fa", "TA", "Gd", "Ex"],
    "ExterQual":   ["Po", "Fa", "TA", "Gd", "Ex"],
    "GarageFinish": ["Unf", "RFn", "Fin"],
    "Functional": ["None", "Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"],
    "BsmtFinType1": ["None", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "BsmtFinType2": ["None", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "BsmtExposure": ["None", "No", "Mn", "Av", "Gd"],
    "LandSlope": ["Gtl", "Mod", "Sev"],
    "Utilities": ["None", "ELO", "NoSeWa", "NoSewr", "AllPub"],
    "LandContour": ["Lvl", "Bnk", "HLS", "Low"],
}

In [ ]:
# Advanced feature engineering transformers for categorical and derived features

class CatEncoder(BaseEstimator, TransformerMixin):
    """Handles both one-hot and ordinal encoding with proper categorical handling.
    Ensures consistent encoding across train/validation/test splits."""
    
    def __init__(self, one_hot_features=None, ordinal_features_levels=None, drop_first=True):
        self.one_hot_features = one_hot_features or []
        self.ordinal_features_levels = ordinal_features_levels or {}
        self.drop_first = drop_first
        self.dummy_cols = None

    def fit(self, X, y=None):
        X = X.copy().fillna("Missing")
        
        # Establishing dummy column structure during fit phase
        ohe = pd.get_dummies(X[self.one_hot_features], drop_first=self.drop_first, dtype='uint8')
        self.dummy_cols = list(ohe.columns)
        return self

    def transform(self, X):
        X = X.copy().fillna("Missing")
        
        # Applying ordinal encoding with predefined hierarchies
        for col, order in self.ordinal_features_levels.items():
            if col in X.columns:
                cat_type = pd.api.types.CategoricalDtype(categories=order, ordered=True)
                X[col] = X[col].astype(cat_type).cat.codes

        # Creating one-hot encoded features
        ohe = pd.get_dummies(X[self.one_hot_features], drop_first=self.drop_first, dtype='uint8')
        
        # Ensuring consistent column structure across datasets
        for col in self.dummy_cols: # type: ignore
            if col not in ohe.columns:
                ohe[col] = 0
        
        ohe = ohe[self.dummy_cols]
        X = X.drop(columns=self.one_hot_features)
        return pd.concat([X, ohe], axis=1)

class TotalArea(BaseEstimator, TransformerMixin):
    """Creates comprehensive area feature by combining basement and ground floor areas."""
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["TotalSF"] = (X["TotalBsmtSF"] + X["GrLivArea"])
        X.drop(columns=["TotalBsmtSF", "GrLivArea"], inplace=True, errors='ignore')
        return X
    
class TotalBaths(BaseEstimator, TransformerMixin):
    """Calculates total bathroom count including partial bathrooms with appropriate weighting."""
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["TotalBaths"] = (X["FullBath"] + 0.5 * X["HalfBath"] + 
                           X["BsmtFullBath"] + 0.5 * X["BsmtHalfBath"])
        X.drop(columns=["FullBath", "HalfBath", "BsmtFullBath", "BsmtHalfBath"], inplace=True, errors='ignore')
        return X
    
class MedianImputer(BaseEstimator, TransformerMixin):
    """Imputes missing numerical values using median strategy for robustness against outliers."""
    
    def __init__(self, columns=None, fill_value=None):
        self.columns = columns if columns is not None else []
        self.fill_value = fill_value
    
    def fit(self, X, y=None):
        if self.fill_value is None:
            valid_columns = [col for col in self.columns if col in X.columns]
            if valid_columns:
                self.fill_value = X[valid_columns].median().to_dict()
            else:
                self.fill_value = {}
        return self

    def transform(self, X):
        X = X.copy()
        if self.fill_value is not None:
            for col in self.columns:
                if col in X.columns and col in self.fill_value:
                    X[col] = X[col].fillna(self.fill_value[col])
        return X

class ToDataFrame(BaseEstimator, TransformerMixin):
    """Ensures output maintains DataFrame structure with proper column names.
    Essential for pipeline compatibility and downstream processing."""
    
    def __init__(self):
        self.columns_ = None

    def fit(self, X, y=None):
        if hasattr(X, "columns"):
            self.columns_ = list(X.columns)
        return self

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            return X
        return pd.DataFrame(X, columns=self.columns_)

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(numerical_values.corr(), linewidth=0.4, annot=True, fmt=".2f", 
            cmap="coolwarm", annot_kws={"size": 6}, cbar_kws={"shrink": .9})

In [ ]:
## Data Preprocessing Pipeline {#preprocessing}

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import set_config
from pandas import DataFrame

# Configuring sklearn to output pandas DataFrames for better data handling
set_config(transform_output="pandas")

# Defining feature imputation strategy for categorical variables
none_fill_features: list = ["BsmtExposure", "BsmtQual", "GarageQual", 
                            "GarageFinish", "GarageType", "GarageCond", "BsmtCond",
                            "FireplaceQu", "MasVnrType", "Exterior1st", "BsmtFinType1",
                            "BsmtFinType2", "Utilities", "Functional", "KitchenQual"  ]

# Features providing minimal predictive value or causing processing issues
columns_to_drop_regardless = ["MiscVal", "Alley", "PoolQC", "BsmtFinType2", "Fence"]

# Numerical pipeline for training data with outlier removal
pipeline_num_train = Pipeline([
    ("drop_columns", dropColumns(columns=["Id"] + columns_to_drop_regardless)),
    ("log_transformer", LogTransformer()),
    ("age_calculator", AgeCalculator()),
    ("outliers_removal", OutliersRemoval()),  # Training-specific outlier removal
    ("year_imputer", YearImputer()),
    ("total_area", TotalArea()),
    ("total_baths", TotalBaths()),
    ("lot_imputer", MedianImputer(["LotFrontage", "MasVnrArea", 
                                   "TotalBaths", "BsmtFinSF1",
                                   "TotalSF", "GarageCars", "BsmtUnfSF",
                                   "BsmtFinSF2", "GarageArea"], None)),
    ("standard_scaler", StandardScaler()),
    ("highly_correlated_features", HighlyCorrelatedFeatures(threshold=0.9)),
])

# Numerical pipeline for validation/test data without outlier removal
pipeline_num_val_test = Pipeline([
    ("drop_columns", dropColumns(columns=["Id"] + columns_to_drop_regardless)),
    ("log_transformer", LogTransformer()),
    ("age_calculator", AgeCalculator()),
    ("year_imputer", YearImputer()),
    ("total_area", TotalArea()),
    ("total_baths", TotalBaths()),
    ("lot_imputer", MedianImputer(["LotFrontage", "MasVnrArea", 
                                   "TotalBaths", "BsmtFinSF1",
                                   "TotalSF", "GarageCars", "BsmtUnfSF",
                                   "BsmtFinSF2", "GarageArea"], None)),
    ("standard_scaler", StandardScaler()),
    ("highly_correlated_features", HighlyCorrelatedFeatures(threshold=0.9)),
])

# Categorical pipeline for training data
pipeline_cat_train = Pipeline([
    ("drop_columns_cat", dropColumns(columns=["Id"] + columns_to_drop_regardless)),
    ("outliers_removal", OutliersRemoval()),  # Consistent outlier handling
    ("feature_imputer", FeatureImputer([], none_fill_features)),
    ("categorical_encoder", CatEncoder(one_hot_features=one_hot_features, 
                                               ordinal_features_levels=ordinal_features_dict)),
])

# Categorical pipeline for validation/test data
pipeline_cat_val_test = Pipeline([
    ("drop_columns_cat", dropColumns(columns=["Id"] + columns_to_drop_regardless)),
    ("feature_imputer", FeatureImputer([], none_fill_features)),
    ("categorical_encoder", CatEncoder(one_hot_features=one_hot_features, 
                                               ordinal_features_levels=ordinal_features_dict)),
])  

# Target variable pipeline ensuring consistent outlier removal
y_train_pipeline = Pipeline([
    ("OutliersRemoval", OutliersRemoval()),])

# Extracting feature sets for pipeline processing
X_num_train = train_data.select_dtypes(include=[np.number]).drop(columns=["SalePrice"], errors='ignore').copy()
X_cat_train = train_data.select_dtypes(include=['object']).copy()
y_train = train_data['SalePrice'].copy()
y_train = y_train_pipeline.fit_transform(y_train)

X_cat_val = X_val.select_dtypes(include=['object']).copy()
X_num_val= X_val.select_dtypes(include=[np.number]).copy()

# Preserving test IDs for submission file creation
test_ids = test_data['Id'].copy()
X_cat_test = test_data.select_dtypes(include=['object']).copy()
X_num_test = test_data.select_dtypes(include=[np.number]).copy()

# Systematic pipeline execution with proper fit/transform strategy
# Training data: fit and transform with outlier removal
num_train = pd.DataFrame(pipeline_num_train.fit_transform(X_num_train))
cat_train = pd.DataFrame(pipeline_cat_train.fit_transform(X_cat_train))

# Fitting validation/test pipelines on training data for consistency
pipeline_num_val_test.fit(X_num_train)
pipeline_cat_val_test.fit(X_cat_train)

# Transforming validation/test data using fitted parameters
num_val = pd.DataFrame(pipeline_num_val_test.transform(X_num_val))
cat_val = pd.DataFrame(pipeline_cat_val_test.transform(X_cat_val))
num_test = pd.DataFrame(pipeline_num_val_test.transform(X_num_test))
cat_test = pd.DataFrame(pipeline_cat_val_test.transform(X_cat_test))

### **Pipeline Strategy for Different Datasets**

When handling train/validation/test data, we need different pipeline strategies:

- Outlier Removal Problem

- Fitting Strategy

- Index Alignment

- Feature Creation Order


In [ ]:
cat_train.info()

In [ ]:
num_train.isnull().sum().sort_values(ascending=False).head(100)

In [ ]:
num_test["TotalBaths"] = num_test["TotalBaths"].fillna(num_test["TotalBaths"].median())
num_test["TotalSF"] = num_test["TotalSF"].fillna(num_test["TotalSF"].median())
num_test["BsmtFinSF2"] = num_test["BsmtFinSF2"].fillna(num_test["BsmtFinSF2"].median())
num_test["BsmtUnfSF"] = num_test["BsmtUnfSF"].fillna(num_test["BsmtUnfSF"].median())

In [ ]:
cat_test = cat_test.loc[:, ~cat_test.columns.duplicated()]


In [ ]:
# Combining numerical and categorical features into unified datasets
train_df  = num_train.join(cat_train)  
val_df  = num_val.join(cat_val)
test_df  = num_test.join(cat_test)

In [ ]:
# Ensuring consistent feature alignment across all datasets
common_columns = list(set(train_df.columns) & set(val_df.columns) & set(test_df.columns))

# Aligning all dataframes to common feature set
train_df = train_df[common_columns].copy()
val_df = val_df[common_columns].copy()
test_df= test_df[common_columns].copy()

In [ ]:
print("shapes:", train_df.shape, val_df.shape, test_df.shape)
print("NaNs?  :", train_df.isnull().any().any(),
                  val_df.isnull().any().any(),
                  test_df.isnull().any().any())

In [ ]:
#Removing any column that is in val/test but not in train: 
for col in val_df.columns:
    if col not in train_df.columns:
        val_df.drop(columns=col, inplace=True, errors='ignore')


In [ ]:
#checking if val_df has some nan values
train_df.isnull().values.any()

In [ ]:
dupes = cat_test.columns[cat_test.columns.duplicated()].unique()
print("duplicate columns:", dupes.tolist())


In [ ]:
test_df.isnull().sum().sort_values(ascending=False).head(50)

In [ ]:
test_df.isnull().sum().sort_values(ascending=False).head(20)

In [ ]:
## Model Implementation {#modeling}

# Establishing baseline performance with linear regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import root_mean_squared_error, r2_score

model_LR = LinearRegression()
model_LR.fit(train_df, y_train)

# Evaluating baseline model performance
y_pred_lr = model_LR.predict(val_df)
rmse_lr = root_mean_squared_error(y_val, y_pred_lr)
r2_lr = r2_score(y_val, y_pred_lr)

print(f" RMSE: {rmse_lr:,.2f}")
print(f" R² Score: {r2_lr:.4f}")

## Model Evaluation and Analysis

### Ensemble Learning with Stacking Regressors

The following stacking implementation goes beyond simple model averaging by learning optimal combination weights through cross-validation. The system automatically discovers which algorithms work best together and how to weight their predictions. 

**Current Ensemble Architecture:**
- **5 Base Learners**: Automatically selected by Optuna from 9 algorithm options
- **Meta-Learning Strategy**: Ridge regression learns optimal weighting of base predictions  
- **Cross-Validation Prevention**: 5-fold CV ensures meta-learner doesn't overfit to base model outputs
- **Algorithm Diversity**: Combines complementary approaches (linear, tree-based, kernel, instance-based)

#### Implementation

**Base Model Selection Process:**
1. *Optuna Optimization*: Binary flags determine which algorithms to include
2. *Hyperparameter Tuning*: Each selected model gets optimized parameters
3. *Ensemble Size Control*: Minimum 2 models required (pruning single-model trials)
4. *Performance Evaluation*: Cross-validation scoring for robust model assessment

**Current Best Combination (Post-Optimization):**
- **Lasso Regression**: Captures linear relationships with feature selection (α=0.227)
- **CatBoost***: Handles categorical features and non-linear patterns (depth=4, lr=0.073)
- **Support Vector Regression**: Complex decision boundaries via kernel methods (C=0.764)
- **Random Forest**: Ensemble variance reduction (170 trees, depth=20)
- **XGBoost**: Extreme gradient boosting with regularization (170 trees, lr=0.073)

**Meta-Learner Configuration:**
- *Ridge Regression*: Optimal base model weighting (α=0.0025)
- *Cross-Validation*: 5-fold CV prevents meta-learner overfitting
- *Automatic Weight Learning*: No manual ensemble weight specification required

#### Performance Evaluation 

**Multi-Metric Assessment:**
- **RMSE**: Primary optimization target (root mean squared error)
- **R² Score**: Variance explanation measurement
- **MAE**: Outlier-robust error assessment (less sensitive to extreme values)

**Validation Approach:**
- **Held-Out Validation**: 20% of training data reserved for final evaluation
- **Outlier Preservation**: Validation/test sets retain outliers for realistic assessment
- **Residual Analysis**: Error distribution examination revealing model behavior on extreme values

This comprehensive evaluation ensures our ensemble generalizes well to unseen data.

In [ ]:
# Importing ensemble learning libraries and evaluation metrics
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as RMSE
from sklearn.metrics import r2_score as R2
from sklearn.model_selection import cross_val_score as CrossValScore
from sklearn.model_selection import KFold 

# Core regression algorithms for ensemble construction
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import (RandomForestRegressor, StackingRegressor,
                              GradientBoostingRegressor)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor as KNR

# Advanced gradient boosting frameworks
from xgboost import XGBRegressor as XGB
from lightgbm import LGBMRegressor as GBM
from catboost import CatBoostRegressor as CB

In [ ]:
# Hyperparameter optimization framework for automated model tuning
import optuna

## Hyperparameter Optimization with Optuna {#optimization}

**What We're Doing: Automated Ensemble Discovery**

Optuna handles the complex task of simultaneously optimizing two critical decisions:
1. **Which models to include** in our stacking ensemble (9 algorithm options)
2. **What hyperparameters** each selected model should use

**The Optimization Process:**
- **Trial-based Search**: Each trial tests a different combination of models and hyperparameters
- **Dynamic Ensemble Composition**: Binary flags (include_lr, include_lasso, etc.) let Optuna decide which algorithms to combine
- **Bayesian Optimization**: Optuna intelligently suggests parameter combinations based on previous trial results
- **Cross-Validation Scoring**: Every ensemble candidate gets evaluated using 5-fold CV to prevent overfitting

**Model Evaluation Strategy:**
- **Objective Function**: Maximizes negative RMSE (converts minimization to maximization problem)
- **Minimum Ensemble Size**: Trials with less than 2 models get pruned (stacking requires multiple base learners)
- **Performance Tracking**: Each trial records which models were selected for later analysis
- **Pruning Strategy**: Poor-performing trials get terminated early to save computation time

**Current Implementation Logic:**
The objective function creates a complete stacking pipeline for each trial, fits it using cross-validation, and returns the mean CV score. This approach ensures our final model selection is robust and generalizable, rather than just optimized for the training set.

In [ ]:
def objective(trial): 
    """Optuna objective function implementing comprehensive hyperparameter optimization.
    Simultaneously optimizes model selection and individual model parameters."""
    
    # Defining hyperparameter search spaces for each algorithm
    catboost_depth = trial.suggest_int("catboost_depth", 3, 10)
    catboost_learning_rate = trial.suggest_float("catboost_learning_rate", 1e-3, 0.3, log=True)
    catboost_iterations = trial.suggest_int("catboost_iterations", 50, 500, step=50)
    catboost_l2_leaf_reg = trial.suggest_float("catboost_l2_leaf_reg", 1e-3, 10.0, log=True)
    ridge_alpha = trial.suggest_float("ridge_alpha", 1e-3, 10.0, log=True)
    lasso_alpha = trial.suggest_float("lasso_alpha", 1e-3, 10.0, log=True)
    elastic_alpha = trial.suggest_float("elastic_alpha", 1e-3, 10.0, log=True)
    elastic_l1_ratio = trial.suggest_float("elastic_l1_ratio", 0.0, 1.0)
    svr_c = trial.suggest_float("svr_c", 1e-3, 10.0, log=True)
    svr_epsilon = trial.suggest_float("svr_epsilon", 1e-3, 1.0, log=True)
    knn_n_neighbors = trial.suggest_int("knn_n_neighbors", 1, 20)
    knn_weights = trial.suggest_categorical("knn_weights", ["uniform", "distance"])
    knn_algorithm = trial.suggest_categorical("knn_algorithm", ["auto", "ball_tree", "kd_tree", "brute"])
    knn_leaf_size = trial.suggest_int("knn_leaf_size", 1, 50)
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 300, step=10)
    rf_max_depth = trial.suggest_int("rf_max_depth", 5, 30, step=5)
    lgb_num_leaves = trial.suggest_int("lgb_num_leaves", 20, 500, step=10)
    lgb_max_depth = trial.suggest_int("lgb_max_depth", 5, 30, step=5)
    
    # Binary model selection flags enabling dynamic ensemble composition
    include_lr = trial.suggest_categorical("include_lr", [True, False])
    include_lasso = trial.suggest_categorical("include_lasso", [True, False])
    include_enet = trial.suggest_categorical("include_enet", [True, False])
    include_catboost = trial.suggest_categorical("include_catboost", [True, False])
    include_svr = trial.suggest_categorical("include_svr", [True, False])
    include_knn = trial.suggest_categorical("include_knn", [True, False])
    include_rf = trial.suggest_categorical("include_rf", [True, False])
    include_lgbm = trial.suggest_categorical("include_lgbm", [True, False])
    include_xgb = trial.suggest_categorical("include_xgb", [True, False])
    
    # Instantiating models with optimized hyperparameters
    CatBoost = CB(depth=catboost_depth,
                  learning_rate=catboost_learning_rate, 
                    iterations=catboost_iterations,
                    l2_leaf_reg=catboost_l2_leaf_reg,
                    random_state=42,)

    SVR_model = SVR(C=svr_c, epsilon=svr_epsilon)
    KNN_model = KNR(n_neighbors=knn_n_neighbors, 
                    weights=knn_weights, 
                    algorithm=knn_algorithm, 
                    leaf_size=knn_leaf_size)
    
    RF_model = RandomForestRegressor(n_estimators=rf_n_estimators, 
                                     max_depth=rf_max_depth, 
                                     random_state=42)
    
    LGBM_model = GBM(num_leaves=lgb_num_leaves,
                    max_depth=lgb_max_depth, 
                    learning_rate=catboost_learning_rate, 
                    random_state=42)
    
    XGB_model = XGB(n_estimators=rf_n_estimators,
                    max_depth=rf_max_depth, 
                    learning_rate=catboost_learning_rate,
                    random_state=42)
    Ridge_model = Ridge(alpha=ridge_alpha, random_state=42)
    Lasso_model = Lasso(alpha=lasso_alpha, random_state=42)
    ElasticNet_model = ElasticNet(alpha=elastic_alpha, 
                                 l1_ratio=elastic_l1_ratio, 
                                 random_state=42)
    
    # Building dynamic estimator list based on selection flags
    estimators = []

    if include_lr:
        estimators.append(("LR", LinearRegression()))
    if include_lasso:
        estimators.append(("Lasso", Lasso_model))
    if include_enet:
        estimators.append(("ElasticNet", ElasticNet_model))
    if include_catboost:
        estimators.append(("CatBoost", CatBoost))
    if include_svr:
        estimators.append(("SVR", SVR_model))
    if include_knn:
        estimators.append(("KNN", KNN_model))
    if include_rf:
        estimators.append(("RF", RF_model))
    if include_lgbm:
        estimators.append(("LGBM", LGBM_model))
    if include_xgb:
        estimators.append(("XGB", XGB_model))
        
    # Ensuring minimum ensemble size for meaningful stacking
    if len(estimators) < 2:
        raise optuna.exceptions.TrialPruned() 
    meta_model = Ridge_model

    # Creating stacking ensemble with dynamic composition
    stacking_model = StackingRegressor(estimators=estimators, 
                                       final_estimator=meta_model, 
                                       cv=5,
                                       n_jobs=-1)
    
    # Evaluating performance through cross-validation
    score = CrossValScore(stacking_model, train_df, y_train,
                          scoring = "neg_root_mean_squared_error",
                          cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs = -1) 
    
    # Recording selected models for analysis
    trial.set_user_attr("used_models", [name for name, _ in estimators])
    return score.mean()

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

In [ ]:
best = study.best_trial
df = study.trials_dataframe(attrs=("number", "value", "params", "user_attrs"))
print("Best model combination:", best.user_attrs["used_models"])

In [ ]:
import optuna.visualization as vis

vis.plot_optimization_history(study)

In [ ]:
vis.plot_param_importances(study)

In [ ]:
print("Best trial:")
trial = study.best_trial
print("  RMSE:", trial.value)
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

I used the same variable names for some parameters, some of them shar they learning rate and so ojn. If we want to be precisem we should separate each parameter for each model used, in this way they get tuned separetely. 

In [ ]:
lasso_model = Lasso(alpha=0.22726857509532503, random_state=42)
catboost_model = CB(depth=4,
                    learning_rate=0.07280162515194599, 
                    iterations=400,
                    l2_leaf_reg=0.006096837447833604,
                    random_state=42)
svr_model = SVR(C=0.7635604502001735, 
                epsilon=0.28940639895970827)
rf_model = RandomForestRegressor(n_estimators=170, 
                                  max_depth=20, 
                                  random_state=42)
lgbm_model = GBM(num_leaves=150,
                    max_depth=25, 
                    learning_rate=0.07280162515194599, 
                    random_state=42)
xgb_model = XGB(n_estimators=170,
                max_depth=170, 
                learning_rate=0.07280162515194599,
                random_state=42)

best_models_combo = [("Lasso", lasso_model), ("CatBoost", catboost_model), ("SVR", svr_model), ("RF", rf_model), ("XGB", xgb_model)]

final_stack = StackingRegressor(
    estimators=best_models_combo,
    final_estimator=Ridge(alpha=0.002527911603259286),
    cv=5,
    n_jobs=-1)

In [ ]:
print(train_df.mean())
print(val_df.mean())
print(y_train.mean(), y_val.mean())

In [ ]:
# Fit the final stacking model on the training data
final_stack.fit(train_df, y_train)
# Predict on the validation set
y_pred_final = final_stack.predict(val_df)

# Calculate RMSE manually by taking the square root of MSE
val_rmse = np.sqrt(RMSE(y_val, y_pred_final))
val_r2 = R2(y_val, y_pred_final)
val_mae = MAE(y_val, y_pred_final)

print("Final Stacking Model Performance on Validation Set:")
print(f"Validation RMSE: {val_rmse:,.2f}")
print(f"Validation R²: {val_r2:.4f}")
print(f"Validation MAE: {val_mae:,.2f}")

In [ ]:
errors = y_val - y_pred_final
plt.hist(errors, bins=50)
plt.title("Residuals Histogram")

Most residuals are centered around 0, suggesting the model is well-calibrated. However, there are some outliers, indicating that the model may struggle with certain data points. This is aalso suggested by MAE being very high compared to the centre of residuals' distribution. MAE is less sensitive to outliers because treats all errors equally, while MSE squares the errors, giving more weight to larger errors. 
This means that MSE is more sensitive to outliers than MAE.


Therefore, MAE should be way less high than MSE, but in this case it is not. This suggests that the model is not performing well on some data points, which may be due to outliers or other factors. In fact, we didn't remove outliers from validation set and test, on porpuse. 

#### Feature Inspection and Importance Analysis

Feature importance analysis provides insights into which features contribute most to model predictions. This helps identify key drivers of house prices and can guide future feature engineering efforts.

In [ ]:
rf_features = rf_model.feature_importances_
lgb_features = lgbm_model.feature_importances_
xgb_features = xgb_model.feature_importances_
catboost_features = catboost_model.feature_importances_
feature_importances = pd.DataFrame({
    "Feature": train_df.columns,
    "RF_Importance": rf_features,
    "LGBM_Importance": lgb_features,
    "XGB_Importance": xgb_features,
    "CatBoost_Importance": catboost_features
})

feature_importances = feature_importances.set_index("Feature")
feature_importances = feature_importances.sort_values(by="RF_Importance", ascending=False)

plt.figure(figsize=(15, 10))
feature_importances.plot(kind="bar", figsize=(15, 10), fontsize=10)
plt.title("Feature Importances from Different Models", fontsize=14)


### Prediction on Test set and creation of submission file for Kaggle

In [ ]:
y_test_pred = final_stack.predict(test_df)

In [ ]:
test_rmse = np.sqrt(RMSE(y_val, y_pred_final))
test_r2 = R2(y_val, y_pred_final)
test_mae = MAE(y_val, y_pred_final)

print("Final Stacking Model Performance on Validation Set:")
print(f"Validation RMSE: {test_rmse:,.2f}")
print(f"Validation R²: {test_r2:.4f}")
print(f"Validation MAE: {test_mae:,.2f}")

In [ ]:
submission = pd.DataFrame({"Id": test_ids,
                           "SalePrice": y_test_pred})
submission.to_csv("submission.csv", index=False)